# Azure Open AI and functions

In [1]:
# %pip install --upgrade openai

In [2]:
import openai
import json
import os
import pytz
import inspect

from datetime import datetime
from dotenv import load_dotenv
from pprint import pprint

In [3]:
load_dotenv("azure.env")

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")

In [4]:
openai.api_version = "2023-07-01-preview"
openai.api_type = "azure"
deployment_name = "gpt-35-turbo-0613"  # Should be the 613

In [5]:
def get_today(location):
    """
    Get current datetime from a location
    """
    try:
        now = datetime.now(pytz.timezone(location))
        today = now.strftime("%d-%b-%Y %H:%M:%S")

        return today
    except:
        return "Error"

In [6]:
get_today("America/New_York")

'06-Sep-2023 04:39:45'

In [7]:
get_today("Europe/Paris")

'06-Sep-2023 10:39:45'

In [8]:
get_today("Europe/London")

'06-Sep-2023 09:39:46'

In [9]:
get_today("Asia/Tokyo")

'06-Sep-2023 17:39:46'

In [10]:
functions = [
    {
        "name": "get_today",
        "description": "Get the current date time in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The location name. The pytz is used to get the timezone for that location. Location names should be in a format like America/New_York, Asia/Bangkok, Europe/London",
                }
            },
            "required": ["location"],
        },
    },
]

available_functions = {
    "get_today": get_today,
}

In [11]:
pprint(functions)

[{'description': 'Get the current date time in a given location',
  'name': 'get_today',
  'parameters': {'properties': {'location': {'description': 'The location '
                                                            'name. The pytz is '
                                                            'used to get the '
                                                            'timezone for that '
                                                            'location. '
                                                            'Location names '
                                                            'should be in a '
                                                            'format like '
                                                            'America/New_York, '
                                                            'Asia/Bangkok, '
                                                            'Europe/London',
                                             'type': 's

In [12]:
# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [13]:
def run_conversation(messages, functions, available_functions, deployment_id):
    # Step 1: send the conversation and available functions to GPT

    response = openai.ChatCompletion.create(
        deployment_id=deployment_id,
        messages=messages,
        functions=functions,
        function_call="auto",
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        print("Recommended Function call:")
        print(response_message.get("function_call"))
        print()

        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        function_name = response_message["function_call"]["name"]

        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]

        # verify function has correct number of arguments
        function_args = json.loads(response_message["function_call"]["arguments"])
        if check_args(function_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = function_to_call(**function_args)

        print("Output of function call:")
        print(function_response)
        print()

        # Step 4: send the info on the function call and function response to GPT

        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "function_call": {
                    "name": response_message["function_call"]["name"],
                    "arguments": response_message["function_call"]["arguments"],
                },
                "content": None,
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in second request:")
        for message in messages:
            print(message)
        print()

        second_response = openai.ChatCompletion.create(
            messages=messages, deployment_id=deployment_id
        )  # get a new response from GPT where it can see the function response

        return second_response

## Testing

In [14]:
messages = [{"role": "user", "content": "What time is it in New York?"}]

assistant_response = run_conversation(
    messages, functions, available_functions, deployment_name
)
print(assistant_response["choices"][0]["message"])

Recommended Function call:
{
  "name": "get_today",
  "arguments": "{\n  \"location\": \"America/New_York\"\n}"
}

Output of function call:
06-Sep-2023 04:40:02

Messages in second request:
{'role': 'user', 'content': 'What time is it in New York?'}
{'role': 'assistant', 'function_call': {'name': 'get_today', 'arguments': '{\n  "location": "America/New_York"\n}'}, 'content': None}
{'role': 'function', 'name': 'get_today', 'content': '06-Sep-2023 04:40:02'}

{
  "role": "assistant",
  "content": "It is currently 4:40 AM in New York. Please note that the time provided is based on the current moment and may slightly differ by the time you read this message."
}


In [15]:
print(assistant_response["choices"][0]["message"]["content"])

It is currently 4:40 AM in New York. Please note that the time provided is based on the current moment and may slightly differ by the time you read this message.


In [16]:
messages = [{"role": "user", "content": "What the current day in Paris?"}]

assistant_response = run_conversation(
    messages, functions, available_functions, deployment_name
)
print(assistant_response["choices"][0]["message"])

Recommended Function call:
{
  "name": "get_today",
  "arguments": "{\n  \"location\": \"Europe/Paris\"\n}"
}

Output of function call:
06-Sep-2023 10:40:09

Messages in second request:
{'role': 'user', 'content': 'What the current day in Paris?'}
{'role': 'assistant', 'function_call': {'name': 'get_today', 'arguments': '{\n  "location": "Europe/Paris"\n}'}, 'content': None}
{'role': 'function', 'name': 'get_today', 'content': '06-Sep-2023 10:40:09'}

{
  "role": "assistant",
  "content": "The current day in Paris is September 6, 2023."
}


In [17]:
print(assistant_response["choices"][0]["message"]["content"])

The current day in Paris is September 6, 2023.


In [18]:
messages = [{"role": "user", "content": "What the current date and time in Paris?"}]

assistant_response = run_conversation(
    messages, functions, available_functions, deployment_name
)
print(assistant_response["choices"][0]["message"])

Recommended Function call:
{
  "name": "get_today",
  "arguments": "{\n\"location\": \"Europe/Paris\"\n}"
}

Output of function call:
06-Sep-2023 10:40:11

Messages in second request:
{'role': 'user', 'content': 'What the current date and time in Paris?'}
{'role': 'assistant', 'function_call': {'name': 'get_today', 'arguments': '{\n"location": "Europe/Paris"\n}'}, 'content': None}
{'role': 'function', 'name': 'get_today', 'content': '06-Sep-2023 10:40:11'}

{
  "role": "assistant",
  "content": "The current date and time in Paris, France is September 6, 2023, 10:40 AM."
}


In [19]:
print(assistant_response["choices"][0]["message"]["content"])

The current date and time in Paris, France is September 6, 2023, 10:40 AM.


In [20]:
messages = [
    {
        "role": "user",
        "content": "What the current date and time in NYC? Format the time with seconds",
    }
]

assistant_response = run_conversation(
    messages, functions, available_functions, deployment_name
)
print(assistant_response["choices"][0]["message"])

Recommended Function call:
{
  "name": "get_today",
  "arguments": "{\n  \"location\": \"America/New_York\"\n}"
}

Output of function call:
06-Sep-2023 04:40:15

Messages in second request:
{'role': 'user', 'content': 'What the current date and time in NYC? Format the time with seconds'}
{'role': 'assistant', 'function_call': {'name': 'get_today', 'arguments': '{\n  "location": "America/New_York"\n}'}, 'content': None}
{'role': 'function', 'name': 'get_today', 'content': '06-Sep-2023 04:40:15'}

{
  "role": "assistant",
  "content": "The current date and time in NYC is September 6, 2023, 04:40:15 (HH:MM:SS)."
}


In [21]:
print(assistant_response["choices"][0]["message"]["content"])

The current date and time in NYC is September 6, 2023, 04:40:15 (HH:MM:SS).


In [24]:
messages = [
    {
        "role": "user",
        "content": "I am in Paris. I will drive for 3 hours. What is my arrival time?",
    }
]

assistant_response = run_conversation(
    messages, functions, available_functions, deployment_name
)
print(assistant_response["choices"][0]["message"])

Recommended Function call:
{
  "name": "get_today",
  "arguments": "{\n  \"location\": \"Europe/Paris\"\n}"
}

Output of function call:
06-Sep-2023 10:40:45

Messages in second request:
{'role': 'user', 'content': 'I am in Paris. I will drive for 3 hours. What is my arrival time?'}
{'role': 'assistant', 'function_call': {'name': 'get_today', 'arguments': '{\n  "location": "Europe/Paris"\n}'}, 'content': None}
{'role': 'function', 'name': 'get_today', 'content': '06-Sep-2023 10:40:45'}

{
  "role": "assistant",
  "content": "If you are in Paris and you drive for 3 hours, your arrival time would be 3 hours from now. \n\nConsidering the current time is 10:40 AM, your estimated arrival time would be around 1:40 PM."
}


In [25]:
print(assistant_response["choices"][0]["message"]["content"])

If you are in Paris and you drive for 3 hours, your arrival time would be 3 hours from now. 

Considering the current time is 10:40 AM, your estimated arrival time would be around 1:40 PM.


In [26]:
messages = [{"role": "user", "content": "I am in Paris. What was the yesterday date?"}]

assistant_response = run_conversation(
    messages, functions, available_functions, deployment_name
)
print(assistant_response["choices"][0]["message"])

Recommended Function call:
{
  "name": "get_today",
  "arguments": "{\n\"location\": \"Europe/Paris\"\n}"
}

Output of function call:
06-Sep-2023 10:41:05

Messages in second request:
{'role': 'user', 'content': 'I am in Paris. What was the yesterday date?'}
{'role': 'assistant', 'function_call': {'name': 'get_today', 'arguments': '{\n"location": "Europe/Paris"\n}'}, 'content': None}
{'role': 'function', 'name': 'get_today', 'content': '06-Sep-2023 10:41:05'}

{
  "role": "assistant",
  "content": "Yesterday's date in Paris would have been September 5, 2023."
}


In [27]:
print(assistant_response["choices"][0]["message"]["content"])

Yesterday's date in Paris would have been September 5, 2023.


In [28]:
messages = [{"role": "user", "content": "I am in Paris. What is the number of days starting from the 1st of January 2023"}]

assistant_response = run_conversation(
    messages, functions, available_functions, deployment_name
)
print(assistant_response["choices"][0]["message"])

Recommended Function call:
{
  "name": "get_today",
  "arguments": "{\n  \"location\": \"Europe/Paris\"\n}"
}

Output of function call:
06-Sep-2023 10:41:47

Messages in second request:
{'role': 'user', 'content': 'I am in Paris. What is the number of days starting from the 1st of January 2023'}
{'role': 'assistant', 'function_call': {'name': 'get_today', 'arguments': '{\n  "location": "Europe/Paris"\n}'}, 'content': None}
{'role': 'function', 'name': 'get_today', 'content': '06-Sep-2023 10:41:47'}

{
  "role": "assistant",
  "content": "As of today, which is September 6, 2023, the number of days starting from January 1, 2023, would be 249 days."
}


In [29]:
print(assistant_response["choices"][0]["message"]["content"])

As of today, which is September 6, 2023, the number of days starting from January 1, 2023, would be 249 days.
